<a href="https://colab.research.google.com/github/nimeshayasith/Computer_vision_Assignment/blob/main/4624_practise_work_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

# --- 1. MOUNT DRIVE ---
try:
    drive.mount('/content/drive', force_remount=True)
    project_path = '/content/drive/MyDrive/CV_Assessment_01'
    if os.path.exists(project_path): os.chdir(project_path)
    print(f"Successfully mounted Google Drive and changed directory to {project_path}")
except ValueError as e:
    print(f"Error mounting Google Drive: {e}. Please ensure you authorize the connection promptly.")
    print("Cannot proceed without Google Drive. Please re-run the cell and authorize.")
    # Exit or raise to prevent further execution if drive is critical
    exit() # Or handle as appropriate

# ==========================================
# 1. THE ADAPTIVE "DOUBLE-CLAHE" PIPELINE
# ==========================================
def segment_blood_vessels(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if img is None: return None, None

    original_shape = img.shape[:2]
    img_resized = cv2.resize(img, (800, 800))

    b, g, r = cv2.split(img_resized)

    _, fov_mask = cv2.threshold(g, 10, 255, cv2.THRESH_BINARY)
    kernel_erode = np.ones((15, 15), np.uint8)
    fov_mask_eroded = cv2.erode(fov_mask, kernel_erode, iterations=2)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    g_clahe = clahe.apply(g)

    background = cv2.medianBlur(g_clahe, 45)
    vessels_diff = cv2.subtract(background, g_clahe)

    clahe_diff = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    vessels_enhanced = clahe_diff.apply(vessels_diff)

    valid_pixels = vessels_enhanced[fov_mask_eroded > 0]
    if len(valid_pixels) > 0:
        mean_val = np.mean(valid_pixels)
        std_val = np.std(valid_pixels)
        dynamic_thresh = mean_val + (1.2 * std_val)
    else:
        dynamic_thresh = 20

    _, binary_vessels = cv2.threshold(vessels_enhanced, dynamic_thresh, 255, cv2.THRESH_BINARY)

    vessels_masked = cv2.bitwise_and(binary_vessels, fov_mask_eroded)

    kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    vessels_closed = cv2.morphologyEx(vessels_masked, cv2.MORPH_CLOSE, kernel_close)

    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(vessels_closed, connectivity=8)
    final_mask_small = np.zeros_like(vessels_closed)

    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] > 20:
            final_mask_small[labels == i] = 255

    final_mask = cv2.resize(final_mask_small, (original_shape[1], original_shape[0]), interpolation=cv2.INTER_NEAREST)

    return img, final_mask

# ==========================================
# 2. VALIDATION METRICS
# ==========================================
def calculate_metrics(predicted_mask, ground_truth_mask):
    S = predicted_mask > 128
    G = ground_truth_mask > 128

    intersection = np.logical_and(S, G).sum()
    union = np.logical_or(S, G).sum()
    sum_S = S.sum()
    sum_G = G.sum()

    dice = (2.0 * intersection) / (sum_S + sum_G) if (sum_S + sum_G) != 0 else 0.0
    jaccard = intersection / union if union != 0 else 0.0
    return dice, jaccard

# ==========================================
# 3. BATCH VALIDATION SCRIPT
# ==========================================
base_path = 'Dataset/IPCV_ ASSIGNMENT_01_DATABASE/Database_For_Practical_Part/For Validation'
img_dir = os.path.join(base_path, 'Foundus Images For Validation')
gt_dir = os.path.join(base_path, 'Ground Truth For Validation')

if not os.path.exists(img_dir) or not os.path.exists(gt_dir):
    print("Error: Could not find validation folders. Check your paths!")
else:
    img_files = sorted([f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png', '.tif'))])
    gt_files = sorted([f for f in os.listdir(gt_dir) if f.endswith(('.jpg', '.png', '.tif'))])

    print(f"Found {len(img_files)} images. Running Advanced Double-CLAHE Pipeline...\n")
    all_dice_scores = []
    all_jaccard_scores = []

    for i, (img_name, gt_name) in enumerate(zip(img_files, gt_files)):
        img_path = os.path.join(img_dir, img_name)
        gt_path = os.path.join(gt_dir, gt_name)

        _, pred_mask = segment_blood_vessels(img_path)
        gt_mask = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

        if pred_mask is not None and gt_mask is not None:
            if pred_mask.shape != gt_mask.shape:
                pred_mask = cv2.resize(pred_mask, (gt_mask.shape[1], gt_mask.shape[0]), interpolation=cv2.INTER_NEAREST)

            dice, jaccard = calculate_metrics(pred_mask, gt_mask)
            all_dice_scores.append(dice)
            all_jaccard_scores.append(jaccard)

            if (i + 1) % 10 == 0:
                print(f"Processed {i + 1}/100 images... Current Avg DSC: {np.mean(all_dice_scores):.4f}")

    avg_dice = np.mean(all_dice_scores)
    avg_jaccard = np.mean(all_jaccard_scores)

    print("\n==============================")
    print("   FINAL VALIDATION RESULTS   ")
    print("==============================")
    print(f"Total Images Processed: {len(all_dice_scores)}")
    print(f"Average Dice Similarity Coefficient (DSC): {avg_dice:.4f}")
    print(f"Average Jaccard Index (IoU): {avg_jaccard:.4f}")
    print("==============================\n")

# ==========================================
# 4. EXPORT 10 IMAGES FOR LATEX REPORT
# ==========================================
    output_folder = '/content/drive/MyDrive/CV_Assessment_01/LaTeX_Report_Images'
    os.makedirs(output_folder, exist_ok=True)

    print(f"\nNow exporting 10 image sets to:\n{output_folder}\n")

    for i in range(10):
        img_name = img_files[i]
        gt_name = gt_files[i]

        img_path = os.path.join(img_dir, img_name)
        gt_path = os.path.join(gt_dir, gt_name)

        # Run pipeline again just for these 10 to save them
        img, pred_mask = segment_blood_vessels(img_path)
        gt_mask = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

        # Define exact filenames for LaTeX
        orig_filename = os.path.join(output_folder, f"img{i+1}_original.png")
        pred_filename = os.path.join(output_folder, f"img{i+1}_result.png")
        gt_filename = os.path.join(output_folder, f"img{i+1}_gt.png")

        # Save to Google Drive
        cv2.imwrite(orig_filename, img)
        cv2.imwrite(pred_filename, pred_mask)
        cv2.imwrite(gt_filename, gt_mask)

    print("SUCCESS! All 30 report images have been saved to your Google Drive folder.")

Mounted at /content/drive
Successfully mounted Google Drive and changed directory to /content/drive/MyDrive/CV_Assessment_01
Found 100 images. Running Advanced Double-CLAHE Pipeline...

Processed 10/100 images... Current Avg DSC: 0.6993
Processed 20/100 images... Current Avg DSC: 0.7055
Processed 30/100 images... Current Avg DSC: 0.7094
Processed 40/100 images... Current Avg DSC: 0.6983
Processed 50/100 images... Current Avg DSC: 0.7009
Processed 60/100 images... Current Avg DSC: 0.6910
Processed 70/100 images... Current Avg DSC: 0.6917
Processed 80/100 images... Current Avg DSC: 0.6937
Processed 90/100 images... Current Avg DSC: 0.6880
Processed 100/100 images... Current Avg DSC: 0.6828

   FINAL VALIDATION RESULTS   
Total Images Processed: 100
Average Dice Similarity Coefficient (DSC): 0.6828
Average Jaccard Index (IoU): 0.5258


Now exporting 10 image sets to:
/content/drive/MyDrive/CV_Assessment_01/LaTeX_Report_Images

SUCCESS! All 30 report images have been saved to your Google D